# Text Mining Exercise

In this exercise we'll work with the `biopython` package to recolect some information of articles related with *medulloblastoma* and published on 2012. To access at these articles we have to conect with databases of the National Center for Biotechnology Information (NCBI). There, we can find databases such as the PubMed database. For this exercise I have based my code on the [biopython documentation](https://biopython.org/docs/latest/Tutorial/chapter_entrez.html#). 

In [1]:
from Bio import Entrez, Medline
import pandas as pd
import sqlite3

As we can see we have load two tools of the `biopython` package, lets see what does each one:

-   `Entrez` is an API of the NCBI so other programs (as mine) can acces, search, and download data. The outputs of `Entrez` are tippically in XML (Extensible Markup Language) format, and `Bio.Entrez` parse the XML output into a python object. Thus, for this exercise we'll need `Bio.Entrez` to collect the medline abstracts on *medulloblastoma*. 

-   `Medline` is the principal database on biomedical articles, biopython has this tool to read easily this type of data and store it in python objects with an specific format.

In [32]:
# We set our email so the NCBI knows who I am and can contact for any problem.s
Entrez.email = "mciudadbosch@gmail.com"

In [33]:
"""
We search on PubMed for articles published in 2012 and with the abstract related on "medollublastoma" published on the 2012. 
"""

stream = Entrez.esearch(db     = "pubmed", 
                        term   = "medulloblastoma[abstract] AND 2012[PDAT]",
                        retmax = 300) 
# NOTE: We tryed with two retmax and it returns the same number of articles.

record = Entrez.read(stream)

In [34]:
id_list = record['IdList']
print(id_list[0:10])

['24049850', '24273611', '23864912', '23691470', '23430850', '23389376', '23379186', '23355800', '23354632', '23330355']


In [35]:
medline_art = Entrez.efetch(db   = "pubmed",
                            id   = id_list,      # We charge the previous articles that we found
                            rettype = "medline", # Download the data in the medline format
                            retmode = "text")

In [37]:
records = Medline.parse(medline_art)
records = list(records)

In [38]:
data = []
for r in records:
    data.append({
        "PMID": r.get("PMID", ""),
        "Title": r.get("TI", ""),
        "Authors": "; ".join(r.get("AU", [])),
        "Journal": r.get("JT", ""),
        "Year": r.get("DP", ""),
        "Abstract": r.get("AB", "")
    })


In [43]:
df_dict = {
        "PMID": [],
        "Title": [],
        "Authors": [],
        "Journal":[],
        "Year": [],
        "Abstract":[]
    }
for r in data: 
    df_dict["PMID"].append(r["PMID"])
    df_dict["Title"].append(r["Title"])
    df_dict["Authors"].append(r["Authors"])
    df_dict["Journal"].append(r["Journal"])
    df_dict["Year"].append(r["Year"])
    df_dict["Abstract"].append(r["Abstract"])

df = pd.DataFrame(df_dict)

In [44]:
print(df.head())

       PMID                                              Title  \
0  24049850        Small-molecule antagonists of Gli function.   
1  24273611  Role of Epidermal Growth Factor-Triggered PI3K...   
2  23864912  Update on molecular and genetic alterations in...   
3  23691470  Adult medulloblastoma associated with syringom...   
4  23430850  Onset of adreno-leukodystrophy after medullobl...   

                                             Authors  \
0  Ardecky R; Magnuson GK; Zou J; Ganji SR; Brown...   
1    Dudu V; Able RA Jr; Rotari V; Kong Q; Vazquez M   
2                    Kool M; Korshunov A; Pfister SM   
3                                            Wang CC   
4  Deib G; Poretti A; Meoded A; Cohen KJ; Raymond...   

                                 Journal      Year  \
0                                             2010   
1  Cellular and molecular bioengineering  2012 Dec   
2                                   Memo  2012 Sep   
3              Cancer biology & medicine  2012 Jun

In [45]:
df.to_csv("medollublastoma_2012_medline.csv", index=False)

## Build a SQLite Data Base

In [24]:
# Create the Data Base if it Doesn't Exist
name_db = 'Medollublastoma-2012-Medline'
conn = sqlite3.connect(name_db + '.db')

In [25]:
df = pd.read_csv("medollublastoma_2012_medline.csv")
print(df.columns)

Index(['PMID', 'Title', 'Authors', 'Journal', 'Year', 'Abstract'], dtype='object')


In [26]:
df.head()

PMID                                              Title  \
0  24049850        Small-molecule antagonists of Gli function.   
1  24273611  Role of Epidermal Growth Factor-Triggered PI3K...   
2  23864912  Update on molecular and genetic alterations in...   
3  23691470  Adult medulloblastoma associated with syringom...   
4  23430850  Onset of adreno-leukodystrophy after medullobl...   

                                             Authors  \
0  Ardecky R; Magnuson GK; Zou J; Ganji SR; Brown...   
1    Dudu V; Able RA Jr; Rotari V; Kong Q; Vazquez M   
2                    Kool M; Korshunov A; Pfister SM   
3                                            Wang CC   
4  Deib G; Poretti A; Meoded A; Cohen KJ; Raymond...   

                                 Journal      Year  \
0                                    NaN      2010   
1  Cellular and molecular bioengineering  2012 Dec   
2                                   Memo  2012 Sep   
3              Cancer biology & medicine  2012 Jun   
4                           JIMD reports      2012   

                                            Abstract  
0  As cancer treatments have shifted toward targe...  
1  Medulloblastoma (MB) is the most common brain ...  
2  Medulloblastoma encompasses a group of aggress...  
3  The association between cerebellar medulloblas...  
4  X-linked adreno-leukodystrophy (ALD) is a pero...

In [27]:
# Creating a cursor object to execute SQL queries on a database table
cursor = conn.cursor()

# Creation table query
query_create_table_1 = """CREATE TABLE IF NOT EXISTS articles(PMID INTEGER PRIMARY KEY AUTOINCREMENT,
                                                              Title TEXT,
                                                              Authors TEXT,
                                                              Journal TEXT, 
                                                              Year YEAR, 
                                                              Abstract TEXT);"""

# Execute the query ¡
cursor.execute(query_create_table_1)

In [28]:
df.to_sql('articles', con=conn, if_exists='replace', index=False)  

# We remove articles that aren't in 2012
conn.execute("DELETE FROM articles WHERE Year NOT LIKE '2012%'")

In [29]:
# We create a table for the authors
query_create_table_2 = """CREATE TABLE IF NOT EXISTS authors (id INTEGER PRIMARY KEY AUTOINCREMENT, 
                                                              Name TEXT, 
                                                              Surname TEXT);"""
conn.execute(query_create_table_2 )

# We create a table with a N-N relation with authors 
query_create_table_3 = """CREATE TABLE IF NOT EXISTS article_authors (article_PMID INTEGER, 
                                                                      author_id INTEGER, 
                                                                      FOREIGN KEY(article_PMID) REFERENCES articles(PMID), 
                                                                      FOREIGN KEY(author_id) REFERENCES authors(id), 
                                                                      PRIMARY KEY(article_PMID, author_id));"""
conn.execute(query_create_table_3)

In [30]:
# Clean Tables (to avoid errors in multiple runs)
conn.execute("DELETE FROM authors;")
conn.execute("DELETE FROM article_authors;")

# Complete the tables
authors_added = []
for _, row in df.iterrows():
    pmid = row['PMID']
    authors = list(set(row['Authors'].split('; ')))
    # NOTE: We have noticed that are possible repetitions in author Name and Surname
    #       Thus we remove the repetitions in authors because we can't differentiate them. 
    for a in authors:
        
        parts = a.split(' ')
        name = parts[-1]
        surname = ' '.join(parts[0:(len(parts)-1)])
        if name + surname not in authors_added:
            conn.execute("INSERT INTO authors (Name, Surname) VALUES (?, ?)", (name, surname))
            authors_added.append(name+surname)
        author_id = conn.execute("SELECT id FROM authors WHERE Name=? AND Surname=?", (name, surname)).fetchall()[0][0]
        
        conn.execute("INSERT INTO article_authors (article_PMID, author_id) VALUES (?, ?)", (pmid, author_id))

In [33]:
# We take the articles that have an author with surname 'Shih'.
query_shih = """SELECT article_authors.article_PMID
                FROM article_authors
                RIGHT JOIN authors ON article_authors.author_id = authors.id
                WHERE authors.surname = 'Shih';"""

shih_pmids = conn.execute(query_shih).fetchall()

condition = ','.join([str(s[0]) for s in shih_pmids])

query_shih_titles = f"""SELECT PMID,Title 
                       FROM articles
                       WHERE PMID IN ({condition});"""

articles_title = conn.execute(query_shih_titles).fetchall()
print(articles_title)


[(22832583, 'Dissecting the genomic complexity underlying medulloblastoma.'), (22832581, 'Subgroup-specific structural variation across 1,000 medulloblastoma genomes.')]


In [34]:
conn.commit()
conn.close()